In [ ]:
import pandas as pd
from dependencies.rolling_correlation import run_rolling_corr_calc

def precompute_correlations(universe_name):
    df_universe = pd.read_parquet(f"data/MSCI Europe_timeseries.parquet")
    df_metadata = pd.read_csv(f"data/metadata/{universe_name}_constituent_list.csv", index_col=0)
    
    # Filter for only the required columns
    independent_vars = [col for col in df_universe.columns if "12m_Fwd_PE" in col or "Price_to_Book" in col]
    df_independent = df_universe[independent_vars]
    
    for sedol in df_metadata["SEDOL"]:
        dependent_variable_name = f"{sedol}_Total_1y_Return"
        df_dependent = df_universe[[dependent_variable_name]]
        
        for rolling_wind in [1, 3, 5, 7]:
            df_timeseries, df_stats = run_rolling_corr_calc(
                df_independent, 
                df_dependent, 
                dependent_variable_name,
                run_rolling_corr=True,
                rolling_windows=[rolling_wind]
            )
            
            # Save the precomputed correlations
            df_timeseries.to_parquet(f"data/precomputed/{universe_name}_{sedol}_{rolling_wind}y_timeseries.parquet")
            df_stats.to_parquet(f"data/precomputed/{universe_name}_{sedol}_{rolling_wind}y_stats.parquet")

precompute_correlations("MSCI Europe")

In [ ]:
import pandas as pd
import streamlit as st
import plotly.graph_objects as go
from plotly.colors import qualitative

@st.cache_data
def load_precomputed_data(universe_name, rolling_wind):
    return pd.read_parquet(f"data/precomputed/{universe_name}_{rolling_wind}y_correlations.parquet")

@st.cache_data
def load_universe_data():
    return pd.read_parquet(f"data/MSCI Europe_timeseries.parquet")

# Streamlit app setup
st.info("This page focuses on allowing the user to filter and analyze the selected dataset.")
st.subheader("Rolling Correlation")

# Load metadata and universe data
universe_name = st.session_state.universe_name
df_metadata = pd.read_csv(f"data/metadata/{universe_name}_constituent_list.csv", index_col=0)
df_universe = load_universe_data()

# Company selection
company_list = list(df_metadata["tickerSymbol"] + " SEDOL=" + df_metadata["SEDOL"])
target_ticker = st.selectbox("Target Company Ticker (*required)", company_list, index=0)
target_sedol = target_ticker.split("=")[1]

# Rolling window selection
rolling_wind = st.selectbox("Select Rolling Window", [1, 3, 5, 7], index=3)

# Load precomputed correlation data
df_corr = load_precomputed_data(universe_name, rolling_wind)

# Get the dependent variable data
dependent_variable_name = f"{target_sedol}_Total_1y_Return"
df_dependent = df_universe[[dependent_variable_name]]

# Filter correlation data for the selected dependent variable
df_timeseries = df_corr.loc[dependent_variable_name].dropna()

# Prepare stats dataframe
df_stats = df_timeseries.describe().T
df_stats["SEDOL"] = df_stats.index.str.split("_").str[0]
df_stats["Attribute"] = df_stats.index.str.split("_").str[1:]

# Variable selection and plotting
# (The rest of your code for variable selection, time range filtering, and plotting remains largely the same)
# ...